<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


In [1]:
import random
import io
import pickle
import lxml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

2025-04-17 18:19:15.372494: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 18:19:15.615875: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-17 18:19:15.820381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744924756.012656   63536 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744924756.069424   63536 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744924756.458787   63536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# descargar de textos.info
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

In [3]:
raw_html = urllib.request.urlopen('https://www.textos.info/julio-verne/la-vuelta-al-mundo-en-80-dias/ebook')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

# pasar todo el texto a minúscula
article_text = article_text.lower()

In [4]:
# en article text se encuentra el texto de todo el libro
article_text[:1000]

' en el año 1872, la casa número 7 de saville-row, burlington gardens \r\n—donde murió sheridan en 1814— estaba habitada por phileas fogg, quien a\r\n pesar de que parecía haber tomado el partido de no hacer nada que \r\npudiese llamar la atención, era uno de los miembros más notables y \r\nsingulares del reformclub de londres. por consiguiente, phileas fogg, personaje enigmático y del cual sólo \r\nse sabía que era un hombre muy galante y de los más cumplidos gentlemen \r\nde la alta sociedad inglesa, sucedía a uno de los más grandes oradores \r\nque honran a inglaterra. decíase que se daba un aire a lo byron —su cabeza, se entiende, \r\nporque, en cuanto a los pies, no tenía defecto alguno—, pero a un byron \r\nde bigote y pastillas, a un byron impasible, que hubiera vivido mil años\r\n sin envejecer. phileas fogg, era inglés de pura cepa; pero quizás no había nacido en\r\n londres. jamás se le había visto en la bolsa ni en el banco, ni en \r\nninguno de los despachos mercantiles de 

In [5]:
len(article_text)

399771

In [6]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('full_corpus.csv', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/karen-pal/borges/refs/heads/master/datasets/full_corpus.csv -o full_corpus.csv
        else:
            !wget gull_corpus.csv https://raw.githubusercontent.com/karen-pal/borges/refs/heads/master/datasets/full_corpus.csv

else:
    print("El dataset ya se encuentra descargado")

--2025-04-17 18:22:20--  http://gull_corpus.csv/
Resolving gull_corpus.csv (gull_corpus.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘gull_corpus.csv’
--2025-04-17 18:22:20--  https://raw.githubusercontent.com/karen-pal/borges/refs/heads/master/datasets/full_corpus.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6022847 (5,7M) [text/plain]
Saving to: ‘full_corpus.csv’

full_corpus.csv     100%[===================>]   5,74M  --.-KB/s    in 0,06s   

2025-04-17 18:22:22 (102 MB/s) - ‘full_corpus.csv’ saved [6022847/6022847]

FINISHED --2025-04-17 18:22:22--
Total wall clock time: 1,2s
Downloaded: 1 files, 5,7M in 0,06s (102 MB/s)


In [25]:
df = pd.read_csv('full_corpus.csv').drop(columns=["Unnamed: 0"])
df_borges = df[df.author == 'Jorge Luis Borges']
df_borges
df_borges_1 = df_borges.iloc[2]
df_borges_1.text


'¿Qué soñará el indescifrable futuro? Soñará que Alonso Quijano puede ser don Quijote sin dejar su aldea y sus libros. Soñará que una víspera de Ulises puede ser más pródiga que el poema que narra sus trabajos. Soñará generaciones humanas que no reconocerán el nombre de Ulises. Soñará sueños más precisos que la vigilia de hoy. Soñará que podremos hacer milagros y que no los haremos, porque será más real imaginarlos. Soñará mundos tan intensos que la voz de una sola de sus aves podría matarte. Soñará que el olvido y la memoria pueden ser actos voluntarios, no agresiones o dádivas del azar. Soñará que veremos con todo el cuerpo, como quería Milton desde la sombra de esos tiernos orbes, los ojos. Soñará un mundo sin la máquina y sin esa doliente máquina, el cuerpo. La vida no es un sueño pero puede llegar a ser un sueño, escribe Novalis. FIN'

In [32]:
for fin in df_borges['text'].str[-100:]:
    print(fin)

é también de olvidar. Abel dijo despacio: —Así es. Mientras dura el remordimiento dura la culpa. FIN
nos costaba un poco de trabajo, pero luego nos perfeccionamos tanto que nos perdíamos enseguida. FIN
te máquina, el cuerpo. La vida no es un sueño pero puede llegar a ser un sueño, escribe Novalis. FIN
incapaz de jinetear un bagual, pero le gustan los caballos y los entiende. Es amigo de un indio. FIN
nueve, ocho, siete, seis, cinco, etcétera. Ese número entero es inconcebible; ergo, Dios existe. FIN
 valiente y se puso a pelear contra los soldados junto al desertor Martín Fierro. FIN El aleph, 1949
 valiente y se puso a pelear contra los soldados junto al desertor Martín Fierro. FIN El aleph, 1949
 Geográficas. Suárez Miranda, Viajes de Varones Prudentes, Libro Cuarto, Cap. XLV, Lérida, 1658. FIN
 no se resolvieron A (ya en plena mística)- Francamente no recuerdo si esa noche nos suicidamos. FIN
ue Almotásim es el «hindú» que el estudiante cree haber matado.   FIN Historia de la eterni

In [40]:
df_borges

,link,text,title,metadata,author
144,https://ciudadseva.com/texto/abel-y-cain-borges/,Abel y Caín se encontraron después de la muert...,Abel y Caín,[Minicuento - Texto completo.],Jorge Luis Borges
145,https://ciudadseva.com/texto/adrogue/,"Era muy lindo, un pueblo laberíntico. A veces,...",Adrogué,[Minicuento - Texto completo.],Jorge Luis Borges
146,https://ciudadseva.com/texto/alguien-sonara/,¿Qué soñará el indescifrable futuro? Soñará qu...,Alguien soñará,[Minicuento - Texto completo.],Jorge Luis Borges
147,https://ciudadseva.com/texto/andres-armoa/,Los años le han dejado unas palabras en guaran...,Andrés Armoa,[Minicuento - Texto completo.],Jorge Luis Borges
148,https://ciudadseva.com/texto/argumentum-ornith...,Cierro los ojos y veo una bandada de pájaros. ...,Argumentum ornithologicum,[Minicuento - Texto completo.],Jorge Luis Borges
...,...,...,...,...,...
201,https://ciudadseva.com/texto/tlon-uqbar-orbis-...,I Debo a la conjunción de un espejo y de una e...,"Tlön, Uqbar, Orbis Tertius",[Cuento - Texto completo.],Jorge Luis Borges
202,https://ciudadseva.com/texto/tres-versiones-de...,"En el Asia Menor o en Alejandría, en el segund...",Tres versiones de Judas,[Cuento - Texto completo.],Jorge Luis Borges
203,https://ciudadseva.com/texto/un-problema/,Imaginemos que en Toledo se descubre un papel ...,Un problema,[Minicuento - Texto completo.],Jorge Luis Borges
204,https://ciudadseva.com/texto/un-sueno-2/,En un desierto lugar del Irán hay una no muy a...,Un sueño,[Minicuento - Texto completo.],Jorge Luis Borges


In [41]:

texto_concatenado = ' '.join(df_borges['text'].astype(str))
texto_concatenado = texto_concatenado.replace('FIN', '')

with open('texto_concatenado.csv', 'w', encoding='utf-8') as f:
    f.write(texto_concatenado)
    

In [9]:
df.author.value_counts()

author
Jorge Luis Borges            62
Julio Cortázar               55
Baldomero Lillo              50
Augusto Monterroso           45
Juan José Arreola            45
Alfonso Reyes                37
Enrique Anderson Imbert      36
Mario Benedetti              33
Julio Ramón Ribeyro          27
Clarice Lispector            25
Roberto Arlt                 25
Julio Torri                  24
Felisberto Hernández         15
Luis Vidales                 14
Adolfo Bioy Casares          13
Rubén Darío                  13
Álvaro Mutis                 11
Edmundo Valadés              10
Juan Rodolfo Wilcock         10
Juan Rulfo                   10
Salarrué                      9
Elena Garro                   9
Manuel A. Alonso              9
Alejo Carpentier              9
Juan Bosch                    8
Eduardo Gudiño Kieffer        8
Virgilio Díaz Grullón         7
Silvina Ocampo                7
Andrés Rivera                 7
Rodolfo Walsh                 6
Ricardo Güiraldes             6
P

In [42]:
len(texto_concatenado)

506942

In [43]:
article_text = texto_concatenado

### Elegir el tamaño del contexto

En este caso, como el modelo de lenguaje es por caracteres, todo un gran corpus
de texto puede ser considerado un documento en sí mismo y el tamaño de contexto
puede ser elegido con más libertad en comparación a un modelo de lenguaje tokenizado por palabras y dividido en documentos más acotados.

In [7]:
# seleccionamos el tamaño de contexto
max_context_size = 100

In [8]:
# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.utils import pad_sequences # se utilizará para padding

In [12]:
clean_article_text = article_text.replace('\t', '').replace('\n', '').replace('\r', '')
print(clean_article_text[:1000])

 en el año 1872, la casa número 7 de saville-row, burlington gardens —donde murió sheridan en 1814— estaba habitada por phileas fogg, quien a pesar de que parecía haber tomado el partido de no hacer nada que pudiese llamar la atención, era uno de los miembros más notables y singulares del reformclub de londres. por consiguiente, phileas fogg, personaje enigmático y del cual sólo se sabía que era un hombre muy galante y de los más cumplidos gentlemen de la alta sociedad inglesa, sucedía a uno de los más grandes oradores que honran a inglaterra. decíase que se daba un aire a lo byron —su cabeza, se entiende, porque, en cuanto a los pies, no tenía defecto alguno—, pero a un byron de bigote y pastillas, a un byron impasible, que hubiera vivido mil años sin envejecer. phileas fogg, era inglés de pura cepa; pero quizás no había nacido en londres. jamás se le había visto en la bolsa ni en el banco, ni en ninguno de los despachos mercantiles de la city. ni las dársenas ni los docks de londres 

In [10]:
# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(article_text)

In [11]:
chars_vocab

{'\t',
 '\n',
 '\r',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '¡',
 '«',
 '»',
 '¿',
 'á',
 'è',
 'é',
 'í',
 'ñ',
 'ó',
 'ú',
 'ü',
 '—'}

In [10]:
# la longitud de vocabulario de caracteres es:
len(chars_vocab)

68

In [11]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

###  Tokenizar

In [12]:
# tokenizamos el texto completo
tokenized_text = [char2idx[ch] for ch in article_text]

In [13]:
tokenized_text[:1000]

[2,
 13,
 37,
 2,
 13,
 8,
 2,
 7,
 46,
 10,
 2,
 33,
 57,
 49,
 16,
 5,
 2,
 8,
 7,
 2,
 23,
 7,
 55,
 7,
 2,
 37,
 0,
 26,
 13,
 12,
 10,
 2,
 49,
 2,
 40,
 13,
 2,
 55,
 7,
 18,
 62,
 8,
 8,
 13,
 59,
 12,
 10,
 47,
 5,
 2,
 51,
 50,
 12,
 8,
 62,
 37,
 29,
 19,
 10,
 37,
 2,
 29,
 7,
 12,
 40,
 13,
 37,
 55,
 2,
 35,
 4,
 11,
 40,
 10,
 37,
 40,
 13,
 2,
 26,
 50,
 12,
 62,
 30,
 2,
 55,
 39,
 13,
 12,
 62,
 40,
 7,
 37,
 2,
 13,
 37,
 2,
 33,
 57,
 33,
 1,
 11,
 2,
 13,
 55,
 19,
 7,
 51,
 7,
 2,
 39,
 7,
 51,
 62,
 19,
 7,
 40,
 7,
 2,
 31,
 10,
 12,
 2,
 31,
 39,
 62,
 8,
 13,
 7,
 55,
 2,
 48,
 10,
 29,
 29,
 5,
 2,
 56,
 50,
 62,
 13,
 37,
 2,
 7,
 35,
 4,
 2,
 31,
 13,
 55,
 7,
 12,
 2,
 40,
 13,
 2,
 56,
 50,
 13,
 2,
 31,
 7,
 12,
 13,
 23,
 3,
 7,
 2,
 39,
 7,
 51,
 13,
 12,
 2,
 19,
 10,
 26,
 7,
 40,
 10,
 2,
 13,
 8,
 2,
 31,
 7,
 12,
 19,
 62,
 40,
 10,
 2,
 40,
 13,
 2,
 37,
 10,
 2,
 39,
 7,
 23,
 13,
 12,
 2,
 37,
 7,
 40,
 7,
 2,
 56,
 50,
 13,
 2,
 35,
 4,
 31,
 5

### Organizando y estructurando el dataset

In [14]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

In [15]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

In [16]:
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

In [17]:
tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]

In [18]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

Nótese que estamos estructurando el problema de aprendizaje como *many-to-many*:

Entrada: secuencia de tokens [$x_0$, $x_1$, ..., $x_N$]

Target: secuencia de tokens [$x_1$, $x_2$, ..., $x_{N+1}$]

De manera que la red tiene que aprender que su salida deben ser los tokens desplazados en una posición y un nuevo token predicho (el N+1).

La ventaja de estructurar el aprendizaje de esta manera es que para cada token de target se propaga una señal de gradiente por el grafo de cómputo recurrente, que es mejor que estructurar el problema como *many-to-one* en donde sólo una señal de gradiente se propaga.

En este punto tenemos en la variable `tokenized_sentences` los versos tokenizados. Vamos a quedarnos con un conjunto de validación que utilizaremos para medir la calidad de la generación de secuencias con la métrica de Perplejidad.

In [19]:
X.shape

(359671, 100)

In [20]:
X[0,:10]

array([ 2, 13, 37,  2, 13,  8,  2,  7, 46, 10])

In [21]:
y[0,:10]

array([13, 37,  2, 13,  8,  2,  7, 46, 10,  2])

In [22]:
vocab_size = len(chars_vocab)

# Definiendo el modelo

In [23]:
from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential

El modelo que se propone como ejemplo consume los índices de los tokens y los transforma en vectores OHE (en este caso no entrenamos una capa de embedding para caracteres). Esa transformación se logra combinando las capas `CategoryEncoding` que transforma a índices a vectores OHE y `TimeDistributed` que aplica la capa a lo largo de la dimensión "temporal" de la secuencia.

In [24]:
model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

/home/nacho/.local/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
E0000 00:00:1744918397.244737   46974 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1744918397.246058   46974 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, None, 68)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 200)      │        53,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 68)       │        13,668 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,468 (263.55 KB)

 Trainable params: 67,468 (263.55 KB)

 Non-trainable params: 0 (0.00 B)


### Definir el modelo

Dado que por el momento no hay implementaciones adecuadas de la perplejidad que puedan operar en tiempo de entrenamiento, armaremos un Callback *ad-hoc* que la calcule en cada epoch.

**Nota**: un Callback es una rutina gatillada por algún evento, son muy útiles para relevar datos en diferentes momentos del desarrollo del modelo. En este caso queremos hacer un cálculo cada vez que termina una epoch de entrenamiento.

In [25]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


In [ ]:

class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5, name_model="my_model.keras"):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.history_ppl = history_ppl
      self.patience = patience
      self.name_model = name_model

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        print(self.name_model)
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        self.history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save(self.name_model)
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


: 

### Entrenamiento

In [ ]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=20, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=256)

Epoch 1/20
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 2.4380

2025-04-17 16:39:06.820092: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2143550400 exceeds 10% of free system memory.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(history_ppl) + 1)
sns.lineplot(x=epoch_count,  y=history_ppl)
plt.show()

In [ ]:
# Cargamos el mejor modelo guardado del entrenamiento para hacer inferencia
model = keras.models.load_model('my_model.keras')


### Predicción del próximo caracter

In [ ]:
# Se puede usar gradio para probar el modelo
# Gradio es una herramienta muy útil para crear interfaces para ensayar modelos
# https://gradio.app/

!pip install -q gradio

In [ ]:
import gradio as gr

def model_response(human_text):

    # Encodeamos
    encoded = [char2idx[ch] for ch in human_text.lower() ]
    # Si tienen distinto largo
    encoded = pad_sequences([encoded], maxlen=max_context_size, padding='pre')

    # Predicción softmax
    y_hat = np.argmax(model.predict(encoded)[0,-1,:])


    # Debemos buscar en el vocabulario el caracter
    # que corresopnde al indice (y_hat) predicho por le modelo
    out_word = ''
    out_word = idx2char[y_hat]

    # Agrego la palabra a la frase predicha
    return human_text + out_word

iface = gr.Interface(
    fn=model_response,
    inputs=["textbox"],
    outputs="text")

iface.launch(debug=True)

### Generación de secuencias

In [ ]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

In [ ]:
input_text='habia una vez'

generate_seq(model, input_text, max_length=max_context_size, n_words=30)

###  Beam search y muestreo aleatorio

In [ ]:
# funcionalidades para hacer encoding y decoding

def encode(text,max_length=max_context_size):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

In [ ]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

In [ ]:
# predicción con beam search
salidas = beam_search(model,num_beams=10,num_words=20,input="habia una vez")

In [ ]:
salidas[0]

In [ ]:
# veamos las salidas
decode(salidas[0])